In [25]:
# Mask R-CNN 영역 분할 예제
import cv2
import numpy as np
import matplotlib.pyplot as plt


def draw_box(image, class_id, confidence, left, top, right, bottom):
    # Draw a bounding box.
    image = cv2.rectangle(image, (left, top), (right, bottom), colors[class_id], 2)

    label = f'{classes[class_id]}: {confidence:.2f}'

    labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
    top = max(top, labelSize[1])
    image = cv2.rectangle(image, (left - 1, top - labelSize[1] - baseLine),
                  (left + labelSize[0], top), colors[class_id], -1)
    image = cv2.putText(image, label, (left, top - baseLine), cv2.FONT_HERSHEY_SIMPLEX,
                0.6, (0, 0, 0), 1, cv2.LINE_AA)
    
    return image


# 모델 & 설정 파일
model = "pre-trained/mask_rcnn/frozen_inference_graph.pb"
config = "pre-trained/mask_rcnn/mask_rcnn_inception_v2_coco_2018_01_28.pbtxt"
class_labels = "pre-trained/mask_rcnn/coco_90.names"
confThreshold = 0.6
maskThreshold = 0.3

# 테스트 이미지 파일
image_files = ["dog.jpg", "person.jpg", "sheep.jpg", "kite.jpg", "traffic.jpg"]
images = []

In [26]:
# 네트워크 생성
network = cv2.dnn.readNet(model, config)

if network.empty():
    print("Network open failed!")

# 클래스 이름 불러오기
classes = []
with open(class_labels, "rt") as f:
    classes = f.read().rstrip("\n").split("\n")

# 각각의 클래스에 대한 랜덤한 색상
colors = np.random.uniform(0, 255, (len(classes), 3))ㅇ

# 출력 레이어 이름 받아오기
# for name in layer_names:
#     print(name)
output_layers = list(["detection_out_final", network.getUnconnectedOutLayersNames()[0]])
print(output_layers)

['detection_out_final', 'detection_masks']


In [27]:
# 실행
for file in image_files:
    image = cv2.imread(f"images/{file}")
    
    if image is None:
        print("Image load failed!")
    
    # 블롭 생성
    blob = cv2.dnn.blobFromImage(image, swapRB=True)
    network.setInput(blob)
    # 추론
    boxes, masks = network.forward(output_layers)
    # boxes: shape=(1, 1, 100, 7)
    # masks: shape=(100, 90, 15, 15)
    
    
    HEIGHT, WIDTH = image.shape[:2]
    num_classes = masks.shape[1]  # 90: coco_90.names 클래스 개수
    num_detections = boxes.shape[2]  # 100
    
    boxes_to_draw = []
    for i in range(num_detections):
        box = boxes[0, 0, i]  # shape=(7,)
        mask = masks[i]  # shape=(90, 15, 15)
        score = box[2]  # box: dummy, classId, "conf", x1, y1, x2, y2
        if score > confThreshold:
            class_id = int(box[1])
            
            x1 = int(box[3] * WIDTH)
            y1 = int(box[4] * HEIGHT)
            x2 = int(box[5] * WIDTH)
            y2 = int(box[6] * HEIGHT)
            
            # x1 = max(0, min(x1, WIDTH - 1))
            # y1 = max(0, min(y1, HEIGHT - 1))
            # x2 = max(0, min(x2, WIDTH - 1))
            # y2 = max(0, min(y2, HEIGHT - 1))
            
            boxes_to_draw.append([image, class_id, score, x1, y1, x2, y2])
            class_mask = mask[class_id]
            
            # 객체별 15x15 마스크를 바운딩 박스 크기로 resize한 후, 불투명 컬러로 표시
            class_mask = cv2.resize(class_mask, (x2 - x1 + 1, y2 - y1 + 1))
            mask = (class_mask > maskThreshold)
            
            ROI = image[y1:y2+1, x1:x2+1][mask]
            image[y1:y2+1, x1:x2+1][mask] = (colors[class_id] * 0.7 + ROI * 0.3).astype(np.uint8)
    
    # 객체별 바운딩 박스 그리기 & 클래스 이름 표시
    for box in boxes_to_draw:
        image = draw_box(*box)
    
    # 추론 시간 표시
    t, _ = network.getPerfProfile()
    label = f"Inference time: {t*1000. / cv2.getTickFrequency():.2f} ms"
    image = cv2.putText(image, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 1, cv2.LINE_AA)
    images.append(image)
    
    # cv2.imshow("image", image)
    # cv2.waitKey()

# cv2.destroyAllWindows()
# cv2.waitKey(1)

Image load failed!


[ WARN:0@40871.928] global /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('images/dog.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.5.5) /Users/runner/work/opencv-python/opencv-python/opencv/modules/core/src/matrix_wrap.cpp:1286: error: (-215:Assertion failed) !(m.empty() && fixedType() && fixedSize()) && "Can't reallocate empty Mat with locked layout (probably due to misused 'const' modifier)" in function 'create'


In [ ]:
# pyplot 으로 출력
plt.figure(figsize=(16, 50), facecolor="lightgray")

plt.subplot(511).axis("off")
plt.imshow(images[0][..., ::-1])

plt.subplot(512).axis("off")
plt.imshow(images[1][..., ::-1])

plt.subplot(513).axis("off")
plt.imshow(images[2][..., ::-1])

plt.subplot(514).axis("off")
plt.imshow(images[3][..., ::-1])

plt.subplot(515).axis("off")
plt.imshow(images[4][..., ::-1])

plt.show()